In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def plot_delivery_time_hist(df):
    sns.histplot(df['duration_minutes'], bins=50)
    plt.title('Distribution of Delivery Times (minutes)')
    plt.xlabel('Minutes')
    plt.show()

def plot_geospatial(df):
    plt.scatter(df['delivery_lng'], df['delivery_lat'], alpha=0.1)
    plt.title('Delivery Locations')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

    plt.scatter(df['pickup_lng'], df['pickup_lat'], alpha=0.1)
    plt.title('Pickup Locations')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()

In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

n_rows = 50000

order_ids = [f"order_{i+1:04d}" for i in range(n_rows)]

# Generate random dates in 2024
def random_date_2024():
    start = datetime(2024, 1, 1)
    end = datetime(2024, 12, 31)
    delta_days = (end - start).days
    rand_day = np.random.randint(0, delta_days + 1)
    return start + timedelta(days=rand_day)

# Generate random pickup datetimes (between 8:00 and 18:00) on random dates
pickup_times = []
for _ in range(n_rows):
    base_date = random_date_2024()
    hour = np.random.randint(8, 18)
    minute = np.random.randint(0, 60)
    second = np.random.randint(0, 60)
    pickup_times.append(base_date + timedelta(hours=hour, minutes=minute, seconds=second))

# Generate delivery times with longer durations for 8–9AM and 3–5PM pickups
delivery_times = []
for pickup in pickup_times:
    hour = pickup.hour
    if 8 <= hour < 9:
        # Morning window: 8–9 hours
        duration = np.random.randint(8*60, (9*60) + 1)
    elif 15 <= hour < 17:
        # Afternoon window: 3–5 hours
        duration = np.random.randint(3*60, (5*60) + 1)
    else:
        # Default: 2–6 hours
        duration = np.random.randint(2*60, (6*60) + 1)
    delivery_times.append(pickup + timedelta(minutes=duration))

def random_lat_lng(center_lat, center_lng, delta=0.05):
    lat = center_lat + np.random.uniform(-delta, delta)
    lng = center_lng + np.random.uniform(-delta, delta)
    return lat, lng

pickup_coords = [random_lat_lng(41.8781, -87.6298) for _ in range(n_rows)]  # Example: Chicago
delivery_coords = [random_lat_lng(41.8781, -87.6298) for _ in range(n_rows)]

df = pd.DataFrame({
    'order_id': order_ids,
    'pickup_time': pickup_times,
    'delivery_time': delivery_times,
    'pickup_lat': [lat for lat, lng in pickup_coords],
    'pickup_lng': [lng for lat, lng in pickup_coords],
    'delivery_lat': [lat for lat, lng in delivery_coords],
    'delivery_lng': [lng for lat, lng in delivery_coords],
})

In [4]:
from src.data_prep import clean_data
from src.features import add_time_features, add_distance_feature
from src.models import train_models, predict_models
from src.evaluation import evaluate
from src.experiments import aa_test, ab_test
from sklearn.model_selection import train_test_split

df = clean_data(df)
df = add_time_features(df)
df = add_distance_feature(df)

# plot_delivery_time_hist(df)
# plot_geospatial(df)

In [5]:
features = ['distance_km', 'hour', 'weekday']
features = ['distance_km', 'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']
X = df[features]
y = df['duration_minutes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = train_models(X_train, y_train)
y_preds = predict_models(models, X_test)

print(df.shape)
print(features)
evaluate(
    y_test,
    y_preds,
    [
        'LinearReg',
        'RandomForest',
        'GBM'
    ]
)

aa_test(y_test)
ab_test(y_test, y_preds[0], y_preds[1])  # Compare Linear vs RandomForest

(50000, 15)
['distance_km', 'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']
LinearReg: MAE=67.27, R2=0.33
RandomForest: MAE=53.71, R2=0.56
GBM: MAE=49.91, R2=0.64
A/A Test p-value: 0.258 (should be > 0.05)
A/B Test p-value: 0.000 (significant difference if < 0.05)


['distance_km', 'hour', 'weekday']  
LinearReg: MAE=72.81, R2=0.16  
RandomForest: MAE=53.79, R2=0.55  
GBM: MAE=51.18, R2=0.61  
A/A Test p-value: 0.652 (should be > 0.05)  
A/B Test p-value: 0.000 (significant difference if < 0.05)  

---

['distance_km', 'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']  
LinearReg: MAE=67.29, R2=0.32  
RandomForest: MAE=53.62, R2=0.56  
GBM: MAE=51.21, R2=0.61  
A/A Test p-value: 0.652 (should be > 0.05)  
A/B Test p-value: 0.000 (significant difference if < 0.05)  
